# Functions for Building a Chess Board

## Notation

Board is stored as a dictionary mapping board locations to piece types

Pieces are named:

    K --> King

    Q --> Queen

    R --> Rook

    B --> Bishop

    N --> Knight
    
    P --> Pawn


In [1]:
# import some basic libraries and functions for this tutorial.
import numpy as np
import os
import pydot
from ipython.display import html, svg, display
import os.path as osp
from pydrake.common import findresourceorthrow, temp_directory
from pydrake.geometry import (
    meshcatvisualizer,
    meshcatvisualizerparams,
    role,
    startmeshcat,
)
from pydrake.math import rigidtransform, rollpitchyaw
from pydrake.multibody.meshcat import jointsliders
from pydrake.multibody.parsing import parser
from pydrake.multibody.plant import addmultibodyplantscenegraph
from pydrake.systems.analysis import simulator
from pydrake.systems.framework import diagrambuilder

from pydrake.trajectories import piecewisepolynomial

from pydrake.all import (addmultibodyplantscenegraph, diagrambuilder,
                         findresourceorthrow, generatehtml,
                         inversedynamicscontroller, meshcatvisualizer,
                         meshcatvisualizerparams, multibodyplant, parser,
                         simulator, startmeshcat, trajectorysource)

ModuleNotFoundError: No module named 'numpy'

In [4]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

NameError: name 'StartMeshcat' is not defined

In [146]:
class Board:
    starting_board = {
        'a1': 'RW',
        'b1': 'NW',
        'c1': 'BW',
        'd1': 'QW',
        'e1': 'KW',
        'f1': 'BW',
        'g1': 'NW',
        'h1': 'RW',

        'a2': 'PW',
        'b2': 'PW',
        'c2': 'PW',
        'd2': 'PW',
        'e2': 'PW',
        'f2': 'PW',
        'g2': 'PW',
        'h2': 'PW',

        'a8': 'RB',
        'b8': 'NB',
        'c8': 'BB',
        'd8': 'QB',
        'e8': 'KB',
        'f8': 'BB',
        'g8': 'NB',
        'h8': 'RB',

        'a7': 'PB',
        'b7': 'PB',
        'c7': 'PB',
        'd7': 'PB',
        'e7': 'PB',
        'f7': 'PB',
        'g7': 'PB',
        'h7': 'PB',
    }

    piece_to_fn = {
        'BB': 'Bishop_B.urdf',
        'BW': 'Bishop_W.urdf',

        'KB': 'King_B.urdf',
        'KW': 'King_W.urdf',

        'NB': 'Knight_B.urdf',
        'NW': 'Knight_W.urdf',

        'PB': 'Pawn_B.urdf',
        'PW': 'Pawn_W.urdf',

        'QB': 'Queen_B.urdf',
        'QW': 'Queen_W.urdf',

        'RB': 'Rook_B.urdf',
        'RW': 'Rook_W.urdf'
    }


    model_dir = '../models/'
    board_fn = 'Board.urdf'


    board_spacing = 0.0635  # This is tile spacing in meters (unit of Drake)

    def location_to_coord(self, location):
        """
        Given location in algebraic notation, generate 0-indexed location of
        piece

        Args:
            location (str): Location in algebraic notation.
        """
        letter_to_idx = {
            'a': 0,
            'b': 1,
            'c': 2,
            'd': 3,
            'e': 4,
            'f': 5,
            'g': 6,
            'h': 7
        }

        return letter_to_idx[location[0]], int(location[1]) - 1
        

    def get_xy_location(self, location):
        x_idx, y_idx = self.location_to_coord(location)
        x = self.board_spacing / 2 + self.board_spacing * x_idx
        y = self.board_spacing / 2 + self.board_spacing * y_idx

        # Origin is in middle of board
        x -= self.board_spacing * 4 
        y -= self.board_spacing * 4 

        return x, y

    def make_board(self, board_dict):
        """
        board_dict maps locations in algebraic notiation to peice types.  Locations not
        listed don't have pieces on them.
        """

        for location, piece in board_dict.items():
            x_idx, y_idx = self.location_to_coord(location)
            x = self.board_spacing / 2 + self.board_spacing * x_idx
            y = self.board_spacing / 2 + self.board_spacing * y_idx

            # Origin is in middle of board
            x -= self.board_spacing * 4 
            y -= self.board_spacing * 4 

            # print(location, piece)
            # print(self.location_to_coord(location))
            # print(x, y)

In [152]:

def create_scene(board: Board, sim_time_step=0.0001):
    board_piece_offset = 0.0

    # Clean up MeshCat.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=sim_time_step)
    parser = Parser(plant)

    parser.AddModelFromFile(osp.join(board.model_dir, board.board_fn))
    idx_to_location = {}
    for location, piece in board.starting_board.items():
        name = location + piece  # Very arbitrary, may change later
        idx = parser.AddModelFromFile(osp.join(board.model_dir, board.piece_to_fn[piece]), name)
        idx_to_location[idx] = location
    
    panda_model_file = FindResourceOrThrow(
    "drake/manipulation/models/"
    "franka_description/urdf/panda_arm_hand.urdf")
    panda_model = parser.AddModelFromFile(panda_model_file)

    # Weld the table to the world so that it's fixed during the simulation.
    board_frame = plant.GetFrameByName("board_body")
    plant.WeldFrames(plant.world_frame(), board_frame)

    # Weld robot to table
    panda_base_frame = plant.GetFrameByName('panda_link0')
    panda_transform = RigidTransform(
    RollPitchYaw(np.asarray([0, 0, -np.pi/2])), p=[0, 0.45, 0])    
    
    plant.WeldFrames(plant.world_frame(), panda_base_frame, panda_transform)

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # We use the default context to calculate the transformation of the table
    # in world frame but this is NOT the context the Diagram consumes.
    plant_context = plant.CreateDefaultContext()
    # plant_context = plant.CreateDefaultContext()

    X_WorldBoard= board_frame.CalcPoseInWorld(plant_context)

    for idx, location in idx_to_location.items():
        piece = plant.GetBodyByName("piece_body", idx)
        # benchy = plant.GetBodyByName("benchy_body", name)
        x, y = board.get_xy_location(location)
        X_BoardPiece = RigidTransform(
            RollPitchYaw(np.asarray([0, 0, 0])), p=[x, y, board_piece_offset])
        X_BoardPiece = X_WorldBoard.multiply(X_BoardPiece)
        plant.SetDefaultFreeBodyPose(piece, X_BoardPiece)

    plant.SetDefaultFreeBodyPose(piece, X_BoardPiece)

    # # Finalize the plant after loading the scene.
    # plant.Finalize()

    # q0 = [-1.57, 0.1, 0, -1.2, 0, 1.6, 0, 0, 0]
    q0 = [1.57, 0, 0, 0.0, 0, 0.0, 0, 0, 0]
    t_lst = np.linspace(0, 5, 30)
    q_knots = np.repeat(np.array(q0).reshape(1, -1), 30, axis=0)
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:9].T)
  
    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))

    builder.Connect(q_traj_system.get_output_port(),
        plant.get_actuation_input_port())
    
    # plant.get_actuation_input_port().FixValue(plant_context, np.zeros(9))


    # num_positions = 9
    # # Add controller
    # kp=[100] * num_positions
    # ki=[1] * num_positions
    # kd=[20] * num_positions

    # controller_plant = plant.get_multibody_plant_for_control()
    # panda_controller = builder.AddSystem(
    # InverseDynamicsController(plant, kp, ki, kd, False))
    # panda_controller.set_name("iiwa_controller")

    # builder.Connect(plant.get_state_output_port(panda_model),
    #                 panda_controller.get_input_port_estimated_state())
    # builder.Connect(panda_controller.get_output_port_control(),
    #                 plant.get_actuation_input_port())

    # Add visualizer to visualize the geometries.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))
    # collision = MeshcatVisualizer.AddToBuilder(
    #     builder, scene_graph, meshcat,
    #     MeshcatVisualizerParams(role=Role.kProximity, prefix="collision"))
    
    diagram = builder.Build()
    return diagram, visualizer, plant

In [153]:
sim_time_step = 0.001

t_lst = np.linspace(0, 5, 30)

board = Board()
diagram, visualizer, plant = create_scene(board, sim_time_step)
context = diagram.CreateDefaultContext()
diagram.Publish(context)

SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

plant_context = plant.GetMyMutableContextFromRoot(context)
# q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('panda'))
q0 = [0, 0, 0, 0.0, 0, 0.0, 0, 0, 0]
# plant.get_actuation_input_port().FixValue(plant_context, q0)
print(plant.GetPositions(plant_context, plant.GetModelInstanceByName('panda')))
# https://github.com/RussTedrake/manipulation/blob/master/exercises/trajectories/rrt_planning.ipynb
plant.SetPositions(plant_context, plant.GetModelInstanceByName('panda'), q0)
print(plant.GetPositions(plant_context, plant.GetModelInstanceByName('panda')))


simulator = Simulator(diagram, context)
simulator.Initialize()
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(1)



# # diagram, visualizer = create_scene(sim_time_step)
# simulator = initialize_simulation(diagram, context)

# plant = diagram.GetSubsystemByName("plant")
# context = plant.CreateDefaultContext()
# diagram.Publish(context)


# visualizer.StartRecording()
# simulator.AdvanceTo(5.0)
# visualizer.PublishRecording()

# Run the simulation with a small time step. Try gradually increasing it!

[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [148]:
board = Board()
board.make_board(board.starting_board)